In [2]:
!pip install boto3 sagemaker

   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   -- ------------------------------------- 1.0/15.8 MB 5.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.8/15.8 MB 4.2 MB/s eta 0:00:04
   ------- -------------------------------- 2.9/15.8 MB 4.5 MB/s eta 0:00:03
   --------- ------------------------------ 3.9/15.8 MB 4.5 MB/s eta 0:00:03
   ------------- -------------------------- 5.2/15.8 MB 4.8 MB/s eta 0:00:03
   ---------------- ----------------------- 6.6/15.8 MB 5.2 MB/s eta 0:00:02
   -------------------- ------------------- 8.1/15.8 MB 5.4 MB/s eta 0:00:02
   ------------------------ --------------- 9.7/15.8 MB 5.6 MB/s eta 0:00:02
   --------------------------- ------------ 10.7/15.8 MB 5.6 MB/s eta 0:00:01
   ----------------------------- ---------- 11.8/15.8 MB 5.6 MB/s eta 0:00:01
   --------------------------------- ------ 13.4/15.8 MB 5.6 MB/s eta 0:00:01
   ------------------------------------ --- 14.4/15.8 MB 5.7 MB/s eta 0:00:01
  

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
datasets 3.2.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.3.9 which is incompatible.
datasets 3.2.0 requires multiprocess<0.70.17, but you have multiprocess 0.70.17 which is incompatible.
glueviz 1.0.0 requires glue-vispy-viewers>=1.0.0, but you have glue-vispy-viewers 0.12.2 which is incompatible.
gradio 4.36.1 requires urllib3~=2.0, but you have urllib3 1.26.20 which is incompatible.
mediapipe 0.9.0 requires protobuf<4,>=3.11, but you have protobuf 5.29.3 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.4 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.26.4 which is incompatible.
spacy 3.7.2 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompa

Dependencies

In [ ]:
import sagemaker
import boto3
import pandas as pd
import numpy as np
import os
import logging
from datetime import datetime
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker import image_uris
from sagemaker import hyperparameters
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.serializers import CSVSerializer

c:\Users\Chinnnolla Koteshwar\Downloads\sage_maker\sage\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/29/25 20:49:07] INFO     Found credentials in shared credentials file: ~/.aws/credentials   credentials.py:1278

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Chinnnolla Koteshwar\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
bucket_name="textdemand"
my_region=boto3.session.Session().region_name
print(my_region)

us-east-1


In [7]:
s3=boto3.resource('s3')
if my_region=="us-east-1":
    s3.create_bucket(Bucket=bucket_name)
    print('s')

s


In [46]:
# role_arn = sagemaker.get_execution_role()
import os
role_arn=os.getenv("roel_arn")

In [4]:
data=pd.read_csv('prod_.csv')

In [5]:
from sklearn.model_selection import train_test_split
train_df,test_df= train_test_split(data, test_size = 0.175, random_state=0)

In [13]:
train_df.to_csv('train.csv', index=False, header=None)
test_df.to_csv('test.csv',index=False,header=None)

In [47]:
!aws s3 cp train.csv s3://textdemand/prodDemand/data/
!aws s3 cp test.csv s3://textdemand/prodDemand/data/

upload: ./train.csv to s3://textdemand/prodDemand/data/train.csv
upload: ./test.csv to s3://textdemand/prodDemand/data/test.csv 


In [9]:
train_df

,product,year,expiri duretion in years,month,no. of new products bought,no of products left for selling,no. of products sold,prodects left,no of products to be bought,demand %
97,suger,2020,2.0,2,350,372,300,22,NaN,85.714286
143,santoor,2020,1.0,12,140,155,150,15,NaN,107.142857
152,colgate,2021,2.0,9,100,121,112,21,91.0,112.000000
106,suger,2020,2.0,11,350,362,350,12,NaN,100.000000
201,life boy soap,2021,1.0,10,110,127,117,17,NaN,106.363636
...,...,...,...,...,...,...,...,...,...,...
67,santoor,2019,1.0,8,100,111,106,11,NaN,106.000000
192,life boy soap,2021,1.0,1,100,104,95,4,NaN,95.000000
117,freedom sunflower oil,2020,0.6,10,60,60,55,0,NaN,91.666667
47,freedom sunflower oil,2019,0.6,12,50,63,58,13,NaN,116.000000


In [65]:
bucket_name="textdemand"
training_data='train.csv'
testing_data='test.csv'
data_path='prod_.csv'
proj_name="prodDemand"
data_file="data"
training_path=f"s3://{bucket_name}/{proj_name}/{data_file}/{training_data}"
testing_path=f"s3://{bucket_name}/{proj_name}/{data_file}/{testing_data}"
data_path=f"s3://{bucket_name}/{proj_name}/{data_file}/{data_path}"


In [12]:
print(training_path)

s3://textdemand/prodDemand/data/train.csv


In [66]:
train_in=TrainingInput(training_path, content_type="csv")
test_in=TrainingInput(testing_path, content_type="csv")
data_in=TrainingInput(data_path, content_type="csv")

In [27]:
print(test_in)

In [28]:
model_op = "s3://{}/{}/{}".format(bucket_name, proj_name, "model")
print(model_op)

s3://textdemand/prodDemand/model


In [56]:
import os
role_arn=os.getenv("role_arn")

In [129]:
training_job="demandpred"
svc = SKLearn(
    entry_point="train.py",
    role=role_arn,
    base_job_name=training_job,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    model_dir=model_op,
    hyperparameters={"C":10, "gamma":1},
    sagemaker_session=sagemaker.Session(),
    framework_version='0.23-1',
    py_version='py3'
)

In [158]:
job_name = training_job + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(job_name)

demandpred2025-01-29-19-28-01


In [159]:
svc.fit({"train": data_in}, wait=True, job_name=job_name)

[01/29/25 19:28:02] INFO     SageMaker Python SDK will collect telemetry to help us better  telemetry_logging.py:90
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[01/29/25 19:28:04] INFO     Creating training-job with name: demandpred2025-01-29-19-28-01         session.py:1042

2025-01-29 13:58:06 Starting - Starting the training job...
2025-01-29 13:58:21 Starting - Preparing the instances for training...
2025-01-29 13:59:12 Downloading - Downloading the training image...
2025-01-29 13:59:53 Training - Training image download completed. Training in progress...2025-01-29 14:00:04,901 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-01-29 14:00:04,904 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-01-29 14:00:04,953 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-01-29 14:00:05,152 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-01-29 14:00:05,166 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-01-29 14:00:05,180 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-01-29 14:00:05,191 sagemaker-training-toolkit INFO     Invoking user

In [161]:
svc_trainer=svc.deploy( initial_instance_count=1,instance_type="ml.t2.medium", serializer=CSVSerializer())

[01/29/25 19:30:46] INFO     Creating model with name: demandpred-2025-01-29-14-00-46-328           session.py:4094

[01/29/25 19:30:47] INFO     Creating endpoint-config with name demandpred-2025-01-29-14-00-46-328  session.py:5889

                    INFO     Creating endpoint with name demandpred-2025-01-29-14-00-46-328         session.py:4711

------------!

In [165]:
endpoint_name = svc_trainer.endpoint_name
print(endpoint_name)

demandpred-2025-01-29-14-00-46-328


In [166]:
sgmkr_runtime = boto3.client("runtime.sagemaker")

In [168]:
response = sgmkr_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body="santoor,2021,3"
)
print(response)

{'ResponseMetadata': {'RequestId': '014788bf-dc7d-40bf-954a-249de0260071', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '014788bf-dc7d-40bf-954a-249de0260071', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Wed, 29 Jan 2025 14:09:05 GMT', 'content-type': 'text/html; charset=utf-8', 'content-length': '3', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'text/html; charset=utf-8', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x000002545840D480>}


In [169]:
response['Body'].read().decode()

'150'

In [64]:
text="2,2021,2"

In [171]:
s3_client=boto3.client('s3')

In [183]:
obj = s3_client.get_object( Bucket=bucket_name, Key="prodDemand/data/test.csv")
testDf = pd.read_csv(obj["Body"])


[01/29/25 19:44:42] INFO     Skipping checksum validation. Response did not contain one of the  httpchecksum.py:481
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

In [184]:
testDf=testDf.iloc[:,0:4]

In [ ]:
# testDf.drop(columns=['2.0'],inplace=True)
testDf

In [177]:
import pandas as pd
import io

In [192]:
csv_buffer = io.StringIO()
testDf.to_csv(csv_buffer,header=None, index=None)
text=csv_buffer.getvalue()
testarr=text.split("\r\n")

In [197]:
testarr[1]

'suger,2021,2'

In [204]:
arrTest=[]
for i in range(len(testarr)-1):    
    print(testarr[i])
    response = sgmkr_runtime.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="text/csv", Body=testarr[i]
    )
    arrTest.append(response['Body'].read().decode())
    print(response['Body'].read().decode())

life boy soap,2021,12

suger,2021,2

santoor,2020,5

freedom sunflower oil,2021,3

colgate,2021,7

freedom sunflower oil,2019,9

life boy soap,2021,7

santoor,2019,5

colgate,2019,9

suger,2020,12

freedom sunflower oil,2019,2

colgate,2019,6

colgate,2021,2

santoor,2019,4

freedom sunflower oil,2021,5

santoor,2020,8

gemini  tea powder,2022,3

gemini  tea powder,2022,4

gemini  tea powder,2020,7

freedom sunflower oil,2020,3

colgate,2020,3

life boy soap,2020,6

colgate,2020,4

colgate,2021,12

gemini  tea powder,2019,1

gemini  tea powder,2021,3

gemini  tea powder,2021,5

life boy soap,2020,5

santoor,2020,4

life boy soap,2019,8

suger,2020,9

gemini  tea powder,2021,1

suger,2021,4

gemini  tea powder,2019,7

gemini  tea powder,2019,4

gemini  tea powder,2020,9

colgate,2019,8

colgate,2020,5

gemini  tea powder,2021,7



In [206]:
obj = s3_client.get_object( Bucket=bucket_name, Key="prodDemand/data/test.csv")
testDf1 = pd.read_csv(obj["Body"])

[01/29/25 19:55:26] INFO     Skipping checksum validation. Response did not contain one of the  httpchecksum.py:481
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

In [213]:
for i in range(len(testarr)-1):
    print(arrTest[i],testDf1['118.6666667'][i])

97 100.0
350 107.7142857
98 96.66666667
47 102.0
98 99.0
61 78.33333333
98 100.0
98 114.1176471
62 56.0
350 72.5
37 96.0
58 95.12195122
98 95.55555556
98 95.0
51 94.0
98 92.85714286
43 108.8888889
98 97.5
98 110.0
59 96.66666667
47 98.33333333
98 103.1578947
95 86.0
58 109.0
18 77.77777778
33 126.6666667
41 90.0
102 105.5555556
150 105.3030303
58 52.87356322
250 95.0
47 84.0
250 89.5
98 155.0
13 135.7142857
19 70.0
51 151.6129032
59 104.1666667
41 86.66666667


In [215]:
s3_client=boto3.client("s3")

In [217]:
sgmkr_clnt=boto3.client("sagemaker")

In [218]:
sgmkr_clnt.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'e57d458f-7b46-48aa-8d26-38a97e6c914e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e57d458f-7b46-48aa-8d26-38a97e6c914e',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 29 Jan 2025 14:32:24 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}